In [1]:
import numpy as np 
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score


In [2]:
import nltk
nltk.download("stopwords")

[nltk_data] Error loading stopwords: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>


False

In [3]:
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [4]:
#data preprocesig begins
#loading daatset in pandas dataset
news_dataset=pd.read_csv("train.csv")
news_dataset

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1
...,...,...,...,...,...
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...,0
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...,0
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...,0
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal...",1


In [5]:
news_dataset.shape

(20800, 5)

In [6]:
#1= FN and 0=TN
news_dataset.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [7]:
#replacing the null values wid mt strings
news_dataset=news_dataset.fillna("")
news_dataset

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1
...,...,...,...,...,...
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...,0
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...,0
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...,0
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal...",1


In [8]:
#merging the author name and title
news_dataset['content']=news_dataset['author']+' '+news_dataset['title']
news_dataset

,id,title,author,text,label,content
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1,Darrell Lucus House Dem Aide: We Didn’t Even S...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0,"Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo..."
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1,Consortiumnews.com Why the Truth Might Get You...
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1,Jessica Purkiss 15 Civilians Killed In Single ...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1,Howard Portnoy Iranian woman jailed for fictio...
...,...,...,...,...,...,...
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...,0,Jerome Hudson Rapper T.I.: Trump a ’Poster Chi...
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...,0,"Benjamin Hoffman N.F.L. Playoffs: Schedule, Ma..."
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...,0,Michael J. de la Merced and Rachel Abrams Macy...
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal...",1,"Alex Ansary NATO, Russia To Hold Parallel Exer..."


In [9]:
news=news_dataset.drop(columns='label',axis=1)
news

,id,title,author,text,content
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus House Dem Aide: We Didn’t Even S...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,"Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo..."
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",Consortiumnews.com Why the Truth Might Get You...
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,Jessica Purkiss 15 Civilians Killed In Single ...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,Howard Portnoy Iranian woman jailed for fictio...
...,...,...,...,...,...
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...,Jerome Hudson Rapper T.I.: Trump a ’Poster Chi...
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...,"Benjamin Hoffman N.F.L. Playoffs: Schedule, Ma..."
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...,Michael J. de la Merced and Rachel Abrams Macy...
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal...","Alex Ansary NATO, Russia To Hold Parallel Exer..."


In [10]:
X=news
y=news_dataset['label']
print(X)
print(y)

          id                                              title  \
0          0  House Dem Aide: We Didn’t Even See Comey’s Let...   
1          1  FLYNN: Hillary Clinton, Big Woman on Campus - ...   
2          2                  Why the Truth Might Get You Fired   
3          3  15 Civilians Killed In Single US Airstrike Hav...   
4          4  Iranian woman jailed for fictional unpublished...   
...      ...                                                ...   
20795  20795  Rapper T.I.: Trump a ’Poster Child For White S...   
20796  20796  N.F.L. Playoffs: Schedule, Matchups and Odds -...   
20797  20797  Macy’s Is Said to Receive Takeover Approach by...   
20798  20798  NATO, Russia To Hold Parallel Exercises In Bal...   
20799  20799                          What Keeps the F-35 Alive   

                                          author  \
0                                  Darrell Lucus   
1                                Daniel J. Flynn   
2                             Consortiu

In [11]:
#stemming : it is a process to reduce a  word to its root word
#for ex: actors, actoress,acting-->act
port_stem=PorterStemmer()


In [12]:
def stemming(content):
    stemmed_content=re.sub('[^a-zA-Z]','',content)
    stemmed_content=stemmed_content.lower()
    stemmed_content=stemmed_content.split()                       
    stemmed_content=[port_stem.stem(word)for word in stemmed_content if not word in stopwords.words('english')]
    stemmed_content= ' '.join(stemmed_content)  
    return stemmed_content                       

In [13]:
news_dataset['content']=news_dataset['content'].apply(stemming)

In [14]:
news_dataset['content']

0        darrelllucushousedemaidewedidntevenseecomeysle...
1        danieljflynnflynnhillaryclintonbigwomanoncampu...
2               consortiumnewscomwhythetruthmightgetyoufir
3        jessicapurkisscivilianskilledinsingleusairstri...
4        howardportnoyiranianwomanjailedforfictionalunp...
                               ...                        
20795    jeromehudsonrappertitrumpaposterchildforwhites...
20796    benjaminhoffmannflplayoffsschedulematchupsando...
20797    michaeljdelamercedandrachelabramsmacysissaidto...
20798    alexansarynatorussiatoholdparallelexercisesinb...
20799                          davidswansonwhatkeepsthefal
Name: content, Length: 20800, dtype: object

In [15]:
X=news_dataset['content'].values
y=news_dataset['label'].values

In [16]:
print(X)
print(y)

['darrelllucushousedemaidewedidntevenseecomeysletteruntiljasonchaffetztweetedit'
 'danieljflynnflynnhillaryclintonbigwomanoncampusbreitbart'
 'consortiumnewscomwhythetruthmightgetyoufir' ...
 'michaeljdelamercedandrachelabramsmacysissaidtoreceivetakeoverapproachbyhudsonsbaythenewyorktim'
 'alexansarynatorussiatoholdparallelexercisesinbalkan'
 'davidswansonwhatkeepsthefal']
[1 0 1 ... 0 1 1]


In [17]:
#converting textual data to numerical data
vectorizer=TfidfVectorizer()
vectorizer.fit(X)
X=vectorizer.transform(X)

In [29]:
print(X)
print(y)

  (0, 5208)	1.0
  (1, 4896)	1.0
  (2, 4549)	1.0
  (3, 9741)	1.0
  (4, 7968)	1.0
  (5, 4994)	1.0
  (6, 11988)	1.0
  (7, 1010)	1.0
  (8, 6803)	1.0
  (9, 12883)	1.0
  (10, 85)	1.0
  (11, 4201)	1.0
  (12, 1165)	1.0
  (13, 9095)	1.0
  (14, 1698)	1.0
  (15, 8725)	1.0
  (16, 13016)	1.0
  (17, 17437)	1.0
  (18, 18102)	1.0
  (19, 9269)	1.0
  (20, 14162)	1.0
  (21, 9651)	1.0
  (22, 15084)	1.0
  (23, 12692)	1.0
  (24, 5799)	1.0
  :	:
  (20775, 1824)	1.0
  (20776, 8919)	1.0
  (20777, 408)	1.0
  (20778, 13973)	1.0
  (20779, 18501)	1.0
  (20780, 4165)	1.0
  (20781, 7747)	1.0
  (20782, 18134)	1.0
  (20783, 3726)	1.0
  (20784, 5551)	1.0
  (20785, 1573)	1.0
  (20786, 7464)	1.0
  (20787, 8238)	1.0
  (20788, 9851)	1.0
  (20789, 19648)	1.0
  (20790, 288)	1.0
  (20791, 4863)	1.0
  (20792, 10228)	1.0
  (20793, 16406)	1.0
  (20794, 11851)	1.0
  (20795, 9666)	1.0
  (20796, 2366)	1.0
  (20797, 13127)	1.0
  (20798, 890)	1.0
  (20799, 5474)	1.0
[1 0 1 ... 0 1 1]


In [19]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,stratify=y,random_state=2)


In [22]:
model=LogisticRegression()

In [23]:
#training time
model.fit(X_train,y_train)


LogisticRegression()

In [24]:
#accuracyyy Score on the trainning data
X_train_predictoion=model.predict(X_train)
training_data_accuracy=accuracy_score(X_train_predictoion,y_train)
training_data_accuracy

0.9927283653846154

In [27]:
#accuracyyy Score on the test data
X_test_prediction=model.predict(X_test)
test_data_accuracy=accuracy_score(X_test_prediction,y_test)
test_data_accuracy

0.5329326923076924

In [31]:
input_data=(0.0164,0.0173,0.0347,0.0070,0.0187,0.0671,0.1056,0.0697,0.0962,0.0251,0.0801,0.1056,0.1266,0.0890,0.0198,0.1133,0.2826,0.3234,0.3238,0.4333,0.6068,0.7652,0.9203,0.9719,0.9207,0.7545,0.8289,0.8907,0.7309,0.6896,0.5829,0.4935,0.3101,0.0306,0.0244,0.1108,0.1594,0.1371,0.0696,0.0452,0.0620,0.1421,0.1597,0.1384,0.0372,0.0688,0.0867,0.0513,0.0092,0.0198,0.0118,0.0090,0.0223,0.0179,0.0084,0.0068,0.0032,0.0035,0.0056,0.0040)
input_data_as_numpy_array=np.asarray(input_data)
print(input_data_as_numpy_array)
input_data_reshaped=input_data_as_numpy_array.reshape(1,-1)
print(input_data_reshaped)

[0.0164 0.0173 0.0347 0.007  0.0187 0.0671 0.1056 0.0697 0.0962 0.0251
 0.0801 0.1056 0.1266 0.089  0.0198 0.1133 0.2826 0.3234 0.3238 0.4333
 0.6068 0.7652 0.9203 0.9719 0.9207 0.7545 0.8289 0.8907 0.7309 0.6896
 0.5829 0.4935 0.3101 0.0306 0.0244 0.1108 0.1594 0.1371 0.0696 0.0452
 0.062  0.1421 0.1597 0.1384 0.0372 0.0688 0.0867 0.0513 0.0092 0.0198
 0.0118 0.009  0.0223 0.0179 0.0084 0.0068 0.0032 0.0035 0.0056 0.004 ]
[[0.0164 0.0173 0.0347 0.007  0.0187 0.0671 0.1056 0.0697 0.0962 0.0251
  0.0801 0.1056 0.1266 0.089  0.0198 0.1133 0.2826 0.3234 0.3238 0.4333
  0.6068 0.7652 0.9203 0.9719 0.9207 0.7545 0.8289 0.8907 0.7309 0.6896
  0.5829 0.4935 0.3101 0.0306 0.0244 0.1108 0.1594 0.1371 0.0696 0.0452
  0.062  0.1421 0.1597 0.1384 0.0372 0.0688 0.0867 0.0513 0.0092 0.0198
  0.0118 0.009  0.0223 0.0179 0.0084 0.0068 0.0032 0.0035 0.0056 0.004 ]]
